In [2]:
# This notebook demonstrates how InSDN preprocessing is performed.

import csv
import pandas as pd
import glob
from sklearn.model_selection import train_test_split

path = "../1 dataset"

pd.set_option('display.max_columns', None) #no max columns
pd.set_option('display.max_rows', None) #no max columns

In [3]:
##################################################################
#                                                   READ/LOAD DATA
##################################################################

# If you want to read single CSV file, use this:

df = pd.read_csv(path + "/LR-HR DDoS 2024 Dataset for SDN-Based Networks.csv")

print("Dataset loaded...")

Dataset loaded...


In [13]:
#show all details of the dataframe
# df.info()

In [14]:
# Number shows total rows and total columns available

df.shape

#The output shows a total of 113,407 instances with 25 features including Label.

(113407, 25)

In [15]:
df['Label'].unique()

# Labeling traffic as normal (0) or malicious (1). From https://www.kaggle.com/datasets/abdussalamahmed/lr-hr-ddos-2024-dataset-for-sdn-based-networks

array([0, 1])

In [16]:
#show the dataframe instances
# df.head() #show first 5 rows
df.tail() #show last 5 rows

#df #show first 5 and last 5 rows

,flow_duration,protocol,srcport,dstport,byte_count,packet_count,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Std,Flow Pkts/s,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,Pkt Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Label
113402,71,0,0,0,0,0,2,0,0,0.0,28169.01408,0,0,0,0,0,0,0.0,0,0,0,0.0,-1,-1,1
113403,72,0,0,0,0,0,2,0,0,0.0,27777.77778,0,0,0,0,0,0,0.0,0,0,0,0.0,-1,-1,1
113404,16,0,0,0,0,0,2,0,0,0.0,125000.00000,0,0,0,0,0,0,0.0,0,0,0,0.0,-1,-1,1
113405,20,0,0,0,0,0,2,0,0,0.0,100000.00000,0,0,0,0,0,0,0.0,0,0,0,0.0,-1,-1,1
113406,25,0,0,0,0,0,2,0,0,0.0,80000.00000,0,0,0,0,0,0,0.0,0,0,0,0.0,-1,-1,1


In [17]:
# Display protocol distribution based on label

df.groupby("Label")["protocol"].value_counts().unstack(fill_value=0)

# Label 0 (Normal) has variety of protocols, but label 1 (malicious) only has protocol 0, suggesting OpenFlow encapsulations.

protocol,0,1,6,17
Label,,,,
0,280,1192,20771,20656
1,70508,0,0,0


In [20]:
df['protocol'].value_counts()

#0 - Unknown
#6 - TCP
#1 - ICMP
#17 - UDP

protocol
0     70788
6     20771
17    20656
1      1192
Name: count, dtype: int64

In [21]:
df.describe()

,flow_duration,protocol,srcport,dstport,byte_count,packet_count,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Std,Flow Pkts/s,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,Pkt Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Label
count,1.134070e+05,113407.000000,113407.000000,113407.000000,1.134070e+05,113407.000000,113407.000000,1.134070e+05,1.134070e+05,113407.000000,1.134070e+05,113407.0,113407.000000,113407.0,113407.0,113407.000000,113407.000000,1.134070e+05,113407.000000,113407.000000,113407.000000,113407.000000,113407.0,113407.000000,113407.000000
mean,5.675157e+06,4.205825,13404.874496,3942.364598,3.149522e+05,138.744257,8.995829,1.607594e+03,1.710389e+04,54.413657,6.774320e+04,0.0,0.049433,0.0,0.0,96.797658,152.417223,4.192369e+05,0.010555,0.073320,0.000265,103.779678,-1.0,11279.700830,0.621725
std,2.290923e+07,6.446244,21656.132359,13247.903299,8.064643e+06,1847.267425,156.940094,1.211427e+05,4.905401e+05,424.427145,1.075165e+05,0.0,0.216770,0.0,0.0,1842.412823,3271.544343,7.155128e+06,0.102194,0.260662,0.016262,336.603800,0.0,24353.898653,0.484959
min,-1.540000e+02,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,-1.694915e+04,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-1.0,-1.000000,0.000000
25%,1.700000e+01,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,0.000000e+00,0.000000e+00,0.000000,1.129040e+03,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-1.0,-1.000000,0.000000
50%,3.600000e+01,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,0.000000e+00,0.000000e+00,0.000000,5.405405e+04,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,-1.0,-1.000000,1.000000
75%,3.325000e+03,6.000000,36313.000000,53.000000,3.300000e+01,1.000000,3.000000,3.300000e+01,1.780000e+02,0.000000,1.176471e+05,0.0,0.000000,0.0,0.0,8.000000,24.000000,9.618800e+03,0.000000,0.000000,0.000000,123.750000,-1.0,-1.000000,1.000000
max,1.200000e+08,17.000000,65513.000000,60994.000000,3.581997e+08,63196.000000,34094.000000,3.160000e+07,1.070000e+08,45423.832520,3.000000e+06,0.0,1.000000,0.0,0.0,360448.000000,681904.000000,9.830000e+08,1.000000,1.000000,1.000000,21413.000000,-1.0,65535.000000,1.000000


In [22]:
##################################################################
#                          DATA CLEANING - CHECK FOR MISSING VALUE
##################################################################

# Check all rows for missing value

#df.isna().sum() #check for missing value, or
df.isnull().sum() #check for missing value

flow_duration        0
protocol             0
srcport              0
dstport              0
byte_count           0
packet_count         0
Tot Bwd Pkts         0
TotLen Fwd Pkts      0
TotLen Bwd Pkts      0
Fwd Pkt Len Std      0
Flow Pkts/s          0
Fwd PSH Flags        0
Bwd PSH Flags        0
Fwd URG Flags        0
Bwd URG Flags        0
Fwd Header Len       0
Bwd Header Len       0
Pkt Len Var          0
FIN Flag Cnt         0
SYN Flag Cnt         0
RST Flag Cnt         0
Pkt Size Avg         0
Init Fwd Win Byts    0
Init Bwd Win Byts    0
Label                0
dtype: int64

In [23]:
# Zero means no missing value - all rows is clean

In [24]:
# Check for value count for each class

df["Label"].value_counts() #output shows that the DDoS label has 2 class, so need to combine

Label
1    70508
0    42899
Name: count, dtype: int64

In [25]:
##################################################################
#                      DATA PREPARATION - MANUAL FEATURE SELECTION
##################################################################

In [29]:
#show columns name

df.columns

Index(['flow_duration', 'protocol', 'srcport', 'dstport', 'byte_count',
       'packet_count', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts',
       'Fwd Pkt Len Std', 'Flow Pkts/s', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len',
       'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt',
       'Pkt Size Avg', 'Init Fwd Win Byts', 'Init Bwd Win Byts', 'Label'],
      dtype='object')

In [30]:
# Drop irrelevant features

# List of 6 features to drop
features_to_drop = ['protocol', 'srcport', 'dstport']

# Drop the specified features from the DataFrame
new_df = df.drop(features_to_drop, axis=1)

In [33]:
#show columns name

new_df.columns

Index(['flow_duration', 'byte_count', 'packet_count', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Std', 'Flow Pkts/s',
       'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags',
       'Fwd Header Len', 'Bwd Header Len', 'Pkt Len Var', 'FIN Flag Cnt',
       'SYN Flag Cnt', 'RST Flag Cnt', 'Pkt Size Avg', 'Init Fwd Win Byts',
       'Init Bwd Win Byts', 'Label'],
      dtype='object')

In [34]:
#### Number shows total rows and total columns available

new_df.shape

(113407, 22)

In [37]:
##################################################################
#                         EXPORT THE DATASET IN CSV (BINARY-CLASS)
##################################################################
# LR-HR DDoS 2024 Dataset for SDN-Based Networks.csv

new_df.to_csv('../ds/lr-hr_ddos_2024_binary_cleaned.csv', index=False, encoding="utf-8")